In [39]:
import os
import re
import json
import openai
import pandas as pd
from dotenv import load_dotenv
from itertools import combinations
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

load_dotenv()



True

In [40]:
common_words_path=r"C:\Users\Manideep S\Downloads\L@\SAT Readiness Report\common_words.txt"

In [41]:
def extract_common_words(common_words_path: str) -> list:
    with open(common_words_path, 'r') as f:
        return [line.strip().lower() for line in f if line.strip()]

In [42]:
## NON OPEN AI
# --------------------

# def flatten_description(desc) -> str:
#     """Flattens any structure (str, list, dict) into a clean single string."""
#     if isinstance(desc, str):
#         return desc
#     elif isinstance(desc, list):
#         return ' '.join(flatten_description(item) for item in desc)
#     elif isinstance(desc, dict):
#         return ' '.join(f"{k} {flatten_description(v)}" for k, v in desc.items())
#     else:
#         return str(desc)

# def clean_text(text) -> str:
#     """Flatten + lowercase, strip non-alphanumerics, collapse whitespace."""
#     text = flatten_description(text)
#     text = text.lower()
#     text = re.sub(r'[^a-z0-9\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text


# # --------------------
# # 2) Core comparison
# # --------------------
# def compute_bleu_cosine_scores(folder_path: str):
#     files = [f for f in os.listdir(folder_path) if re.match(r'.*_\d+\.json$', f)]
#     if len(files) < 2:
#         raise ValueError("Need at least two .json files to compare.")
#     # Load and clean each file into a title→description dict
#     all_data = {}
#     for fn in files:
#         with open(os.path.join(folder_path, fn), 'r', encoding='utf-8') as f:
#             arr = json.load(f)
#         d = {}
#         for entry in arr:
#             title = entry['title']
#             desc  = clean_text(entry['description'])
#             d[title] = desc
#         all_data[fn] = d

#     all_sections = sorted({title for d in all_data.values() for title in d.keys()})
#     smoothing = SmoothingFunction().method1
#     rows = []

#     for f1, f2 in combinations(files, 2):
#         row = {'doc1': f1, 'doc2': f2}
#         full1, full2 = [], []

#         for section in all_sections:
#             t1 = all_data[f1].get(section, "")
#             t2 = all_data[f2].get(section, "")
#             common_words = set(extract_common_words(common_words_path))
#             # Remove words in common_words from t1 and t2
#             t1 = " ".join([w for w in t1.split() if w not in common_words])
#             t2 = " ".join([w for w in t2.split() if w not in common_words])
#             print("\n\ntext1:", t1)
#             print("\ntext2:", t2)
#             full1.append(t1)
#             full2.append(t2)

#             bleu = sentence_bleu([t1.split()], t2.split(), smoothing_function=smoothing)
#             vecs = TfidfVectorizer().fit_transform([t1, t2])
#             cos = cosine_similarity(vecs[0:1], vecs[1:2])[0][0]

#             row[f"{section}_BLEU"]   = round(bleu, 4)
#             row[f"{section}_Cosine"] = round(cos, 4)

#         full1 = " ".join(full1)
#         full2 = " ".join(full2)
#         bleu_full = sentence_bleu([full1.split()], full2.split(), smoothing_function=smoothing)
#         vecs_full = TfidfVectorizer().fit_transform([full1, full2])
#         cos_full  = cosine_similarity(vecs_full[0:1], vecs_full[1:2])[0][0]

#         row["FULL_TEXT_BLEU"]   = round(bleu_full, 4)
#         row["FULL_TEXT_Cosine"] = round(cos_full, 4)

#         rows.append(row)

#     return pd.DataFrame(rows)

In [43]:
### OPEN AI

# # --------------------
# # 1) Utility functions
# # --------------------
# def flatten_description(desc) -> str:
#     if isinstance(desc, str):
#         return desc
#     elif isinstance(desc, list):
#         return ' '.join(flatten_description(item) for item in desc)
#     elif isinstance(desc, dict):
#         return ' '.join(f"{k} {flatten_description(v)}" for k, v in desc.items())
#     else:
#         return str(desc)

# def clean_text(text) -> str:
#     text = flatten_description(text)
#     text = text.lower()
#     text = re.sub(r'[^a-z0-9\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# def extract_common_words(common_words_path: str) -> list:
#     with open(common_words_path, 'r') as f:
#         return [line.strip().lower() for line in f if line.strip()]

# def get_embedding(text: str, model: str = "text-embedding-3-small") -> list:
#     response = openai.embeddings.create(model=model, input=text)
#     return response.data[0].embedding

# def compute_cosine_from_embeddings(e1, e2) -> float:
#     return cosine_similarity([e1], [e2])[0][0]

# # --------------------
# # 2) Core comparison
# # --------------------
# def compute_bleu_openai_cosine_scores(folder_path: str, common_words_path: str):
#     files = [f for f in os.listdir(folder_path) if re.match(r'.*_\d+\.json$', f)]
#     if len(files) < 2:
#         raise ValueError("Need at least two `.json` files to compare.")
    
#     # Load and clean each file into a title→description dict
#     all_data = {}
#     for fn in files:
#         with open(os.path.join(folder_path, fn), 'r', encoding='utf-8') as f:
#             arr = json.load(f)
#         d = {}
#         for entry in arr:
#             title = entry['title']
#             desc  = clean_text(entry['description'])
#             d[title] = desc
#         all_data[fn] = d

#     all_sections = sorted({title for d in all_data.values() for title in d.keys()})
#     smoothing = SmoothingFunction().method1
#     rows = []

#     for f1, f2 in combinations(files, 2):
#         row = {'doc1': f1, 'doc2': f2}
#         full1, full2 = [], []

#         for section in all_sections:
#             t1 = all_data[f1].get(section, "")
#             t2 = all_data[f2].get(section, "")
#             common_words = set(extract_common_words(common_words_path))
#             t1 = " ".join([w for w in t1.split() if w not in common_words])
#             t2 = " ".join([w for w in t2.split() if w not in common_words])

#             print("\n\ntext1:", t1)
#             print("\ntext2:", t2)

#             full1.append(t1)
#             full2.append(t2)

#             # BLEU
#             bleu = sentence_bleu([t1.split()], t2.split(), smoothing_function=smoothing)
#             row[f"{section}_BLEU"] = round(bleu, 4)

#             print("\n\nbleu:", bleu)

#             # Embedding-based Cosine
#             e1 = get_embedding(t1)
#             e2 = get_embedding(t2)
#             cos = compute_cosine_from_embeddings(e1, e2)
#             print("\ncosine:", cos)
#             row[f"{section}_Cosine"] = round(cos, 4)

#         joined1 = " ".join(full1)
#         joined2 = " ".join(full2)

#         bleu_full = sentence_bleu([joined1.split()], joined2.split(), smoothing_function=smoothing)
#         e1_full = get_embedding(joined1)
#         e2_full = get_embedding(joined2)
#         cos_full = compute_cosine_from_embeddings(e1_full, e2_full)

#         row["FULL_TEXT_BLEU"] = round(bleu_full, 4)
#         row["FULL_TEXT_Cosine"] = round(cos_full, 4)

#         rows.append(row)

#     return pd.DataFrame(rows)


In [44]:
## OPEN AI JSON PARSING

import os, json, re
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
import openai

# --------------------
# 1) Utility functions
# --------------------
def flatten_description(desc) -> str:
    if isinstance(desc, str):
        return desc
    elif isinstance(desc, list):
        return ' '.join(flatten_description(item) for item in desc)
    elif isinstance(desc, dict):
        return ' '.join(flatten_description(v) for v in desc.values())
    else:
        return str(desc)

def get_embedding(text: str, model: str = "text-embedding-3-small") -> list:
    response = openai.embeddings.create(model=model, input=text)
    return response.data[0].embedding

def compute_cosine_from_embeddings(e1, e2) -> float:
    return cosine_similarity([e1], [e2])[0][0]

# --------------------
# 2) Updated JSON parsing
# --------------------
def extract_sections_with_weekly_schedule(data):
    sections = {}

    def flatten(content):
        if isinstance(content, str):
            return content
        elif isinstance(content, list):
            return " ".join(flatten(item) for item in content)
        elif isinstance(content, dict):
            return " ".join(flatten(v) for v in content.values())
        return ""

    for section in data:
        title = section.get("title", "Untitled")
        desc = section.get("description")

        if title == "Your Study Plan: Based on You" and isinstance(desc, list):
            for subsection in desc:
                sub_title = subsection.get("title")
                sub_desc = subsection.get("description")

                if sub_title == "Study Plan Phases" and isinstance(sub_desc, list):
                    for phase_dict in sub_desc:
                        for phase_name, phase_content in phase_dict.items():
                            sections[phase_name] = flatten(phase_content)
                    guidelines = subsection.get("guidelines", [])
                    if guidelines:
                        sections["Study Plan Guidelines"] = flatten(guidelines)

                elif sub_title == "weekly_schedule":
                    if isinstance(sub_desc, list):
                        sections["weekly_schedule"] = flatten(" ".join(sub_desc))

                else:
                    sections[sub_title] = flatten(sub_desc)
        else:
            sections[title] = flatten(desc)

    return sections

# --------------------
# 3) Core comparison logic
# --------------------
def compute_bleu_openai_cosine_scores(folder_path: str):
    files = [f for f in os.listdir(folder_path) if re.match(r'.*_\d+\.json$', f)]
    if len(files) < 2:
        raise ValueError("Need at least two `.json` files to compare.")

    all_data = {}
    for fn in files:
        with open(os.path.join(folder_path, fn), 'r', encoding='utf-8') as f:
            json_data = json.load(f)

        extracted = extract_sections_with_weekly_schedule(json_data)
        all_data[fn] = extracted

    # Get all unique sections across all files
    all_sections = sorted({title for data in all_data.values() for title in data.keys()})
    smoothing = SmoothingFunction().method1
    rows = []

    for f1, f2 in combinations(files, 2):
        row = {'doc1': f1, 'doc2': f2}
        full1, full2 = [], []

        for section in all_sections:
            t1 = all_data[f1].get(section, "")
            t2 = all_data[f2].get(section, "")

            print(f"\n\nSection: {section}")
            print("Text1:", t1[:300])
            print("Text2:", t2[:300])

            full1.append(t1)
            full2.append(t2)

            bleu = sentence_bleu([t1.split()], t2.split(), smoothing_function=smoothing)
            row[f"{section}_BLEU"] = round(bleu, 4)

            e1 = get_embedding(t1)
            e2 = get_embedding(t2)
            cos = compute_cosine_from_embeddings(e1, e2)
            row[f"{section}_Cosine"] = round(cos, 4)

        joined1 = " ".join(full1)
        joined2 = " ".join(full2)

        bleu_full = sentence_bleu([joined1.split()], joined2.split(), smoothing_function=smoothing)
        e1_full = get_embedding(joined1)
        e2_full = get_embedding(joined2)
        cos_full = compute_cosine_from_embeddings(e1_full, e2_full)

        row["FULL_TEXT_BLEU"] = round(bleu_full, 4)
        row["FULL_TEXT_Cosine"] = round(cos_full, 4)

        rows.append(row)

    return pd.DataFrame(rows)


In [45]:
# ## OPEN AI JSON PARSING

# import os, json, re
# from itertools import combinations
# from sklearn.metrics.pairwise import cosine_similarity
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# import pandas as pd
# import openai

# # --------------------
# # 1) Utility functions
# # --------------------
# def flatten_description(desc) -> str:
#     if isinstance(desc, str):
#         return desc
#     elif isinstance(desc, list):
#         return ' '.join(flatten_description(item) for item in desc)
#     elif isinstance(desc, dict):
#         return ' '.join(f"{k} {flatten_description(v)}" for k, v in desc.items())
#     else:
#         return str(desc)

# def clean_text(text) -> str:
#     text = flatten_description(text)
#     text = text.lower()
#     text = re.sub(r'[^a-z0-9\s]', ' ', text)
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# def extract_common_words(common_words_path: str) -> list:
#     with open(common_words_path, 'r') as f:
#         return [line.strip().lower() for line in f if line.strip()]

# def get_embedding(text: str, model: str = "text-embedding-3-small") -> list:
#     response = openai.embeddings.create(model=model, input=text)
#     return response.data[0].embedding

# def compute_cosine_from_embeddings(e1, e2) -> float:
#     return cosine_similarity([e1], [e2])[0][0]

# # --------------------
# # 2) Updated JSON parsing
# # --------------------
# def extract_sections_with_weekly_schedule(data):
#     sections = {}

#     def flatten(content):
#         if isinstance(content, str):
#             return content
#         elif isinstance(content, list):
#             return " ".join(flatten(item) for item in content)
#         elif isinstance(content, dict):
#             return " ".join(flatten(v) for v in content.values())
#         return ""

#     for section in data:
#         title = section.get("title", "Untitled")
#         desc = section.get("description")

#         if title == "Your Study Plan: Based on You" and isinstance(desc, list):
#             for subsection in desc:
#                 sub_title = subsection.get("title")
#                 sub_desc = subsection.get("description")

#                 if sub_title == "Study Plan Phases" and isinstance(sub_desc, list):
#                     for phase_dict in sub_desc:
#                         for phase_name, phase_content in phase_dict.items():
#                             sections[phase_name] = flatten(phase_content)
#                     guidelines = subsection.get("guidelines", [])
#                     if guidelines:
#                         sections["Study Plan Guidelines"] = flatten(guidelines)

#                 elif sub_title == "weekly_schedule":
#                     # Concatenate all weekly schedule lines into a single string
#                     if isinstance(sub_desc, list):
#                         sections["weekly_schedule"] = flatten(" ".join(sub_desc))

#                 else:
#                     sections[sub_title] = flatten(sub_desc)
#         else:
#             sections[title] = flatten(desc)

#     return sections

# # --------------------
# # 3) Core comparison logic
# # --------------------
# def compute_bleu_openai_cosine_scores(folder_path: str, common_words_path: str):
#     files = [f for f in os.listdir(folder_path) if re.match(r'.*_\d+\.json$', f)]
#     if len(files) < 2:
#         raise ValueError("Need at least two `.json` files to compare.")

#     all_data = {}
#     for fn in files:
#         with open(os.path.join(folder_path, fn), 'r', encoding='utf-8') as f:
#             json_data = json.load(f)

#         extracted = extract_sections_with_weekly_schedule(json_data)
#         cleaned = {k: clean_text(v) for k, v in extracted.items()}
#         all_data[fn] = cleaned

#     all_sections = sorted({title for d in all_data.values() for title in d.keys()})
#     smoothing = SmoothingFunction().method1
#     rows = []

#     for f1, f2 in combinations(files, 2):
#         row = {'doc1': f1, 'doc2': f2}
#         full1, full2 = [], []

#         for section in all_sections:
#             t1 = all_data[f1].get(section, "")
#             t2 = all_data[f2].get(section, "")
#             common_words = set(extract_common_words(common_words_path))
#             t1 = " ".join([w for w in t1.split() if w not in common_words])
#             t2 = " ".join([w for w in t2.split() if w not in common_words])

#             print("\n\nSection:", section)
#             print("Text1:", t1)
#             print("Text2:", t2)

#             full1.append(t1)
#             full2.append(t2)

#             bleu = sentence_bleu([t1.split()], t2.split(), smoothing_function=smoothing)
#             row[f"{section}_BLEU"] = round(bleu, 4)

#             e1 = get_embedding(t1)
#             e2 = get_embedding(t2)
#             cos = compute_cosine_from_embeddings(e1, e2)
#             row[f"{section}_Cosine"] = round(cos, 4)

#         joined1 = " ".join(full1)
#         joined2 = " ".join(full2)

#         bleu_full = sentence_bleu([joined1.split()], joined2.split(), smoothing_function=smoothing)
#         e1_full = get_embedding(joined1)
#         e2_full = get_embedding(joined2)
#         cos_full = compute_cosine_from_embeddings(e1_full, e2_full)

#         row["FULL_TEXT_BLEU"] = round(bleu_full, 4)
#         row["FULL_TEXT_Cosine"] = round(cos_full, 4)

#         rows.append(row)

#     return pd.DataFrame(rows)


In [46]:
##### Single user example to test the code:

# import os
# import pandas as pd
# user = "Aarthi"
# path = r"C:\Users\Manideep S\Downloads\L@\SAT Readiness Report\Users_data\{user}".format(user=user)
# df = pd.DataFrame()


# path_user = path
# if os.path.isdir(path_user):
#     print(f"Processing user folder: {path_user}")
#     try:
#         results = compute_bleu_openai_cosine_scores(path_user)
#         results['User'] = user  # Add a column to track which user's folder it was
#         df = pd.concat([df, results], ignore_index=True)
#     except Exception as e:
#         print(f"Error processing {user}: {e}")

# print(df)

In [47]:
import os
import pandas as pd

path = r"C:\Users\Manideep S\Downloads\L@\SAT Readiness Report\Users_data"
df = pd.DataFrame()

for user in os.listdir(path):
    path_user = os.path.join(path, user)
    if os.path.isdir(path_user):
        print(f"Processing user folder: {path_user}")
        try:
            results = compute_bleu_openai_cosine_scores(path_user)
            results['User'] = user  # Add a column to track which user's folder it was
            df = pd.concat([df, results], ignore_index=True)
        except Exception as e:
            print(f"Error processing {user}: {e}")

Processing user folder: C:\Users\Manideep S\Downloads\L@\SAT Readiness Report\Users_data\Aarthi


Section: Footer
Text1: Safe harbor The scores, assessments, and recommendations provided in this report are for informational purposes only and do not constitute professional educational, psychological, or legal advice. Current performance is not necessarily indicative of actual or future results, as test-day conditions a
Text2: Safe harbor The scores, assessments, and recommendations provided in this report are for informational purposes only and do not constitute professional educational, psychological, or legal advice. Current performance is not necessarily indicative of actual or future results, as test-day conditions a


Section: Math Strategies
Text1: 1. Target Advanced Math by working through incremental problem sets that gradually increase in difficulty, helping you push beyond comfort without disrupting your balance. 2. Refine your Algebra skills by reviewing problem-solving steps

In [48]:
# Define output file path
output_path = os.path.join(path, 'bleu_cosine_results.csv')

# Delete the file if it already exists
if os.path.exists(output_path):
    os.remove(output_path)
    print(f"Deleted existing file: {output_path}")

# Save the new results to CSV
df.to_csv(output_path, index=False)
print(f"Saved results to {output_path}")

Saved results to C:\Users\Manideep S\Downloads\L@\SAT Readiness Report\Users_data\bleu_cosine_results.csv


In [52]:
import pandas as pd
from IPython.display import display

# Calculate the mean for each column (excluding non-numeric columns)
df_avg = df.select_dtypes(include='number').mean().to_frame().T

# Convert to percentages (multiply by 100)
df_avg_percent = df_avg * 100

# Beautiful print


pd.set_option('display.float_format', '{:,.2f}'.format)
display(df_avg_percent)

,Footer_BLEU,Footer_Cosine,Math Strategies_BLEU,Math Strategies_Cosine,Reading Writing Strategies_BLEU,Reading Writing Strategies_Cosine,Sample Weekly Schedule_BLEU,Sample Weekly Schedule_Cosine,Study Plan Phases_BLEU,Study Plan Phases_Cosine,...,Want to Go Further?_BLEU,Want to Go Further?_Cosine,What This Means For You_BLEU,What This Means For You_Cosine,Your Learning Persona_BLEU,Your Learning Persona_Cosine,Your SAT Practice Snapshot_BLEU,Your SAT Practice Snapshot_Cosine,FULL_TEXT_BLEU,FULL_TEXT_Cosine
0,100.00,100.00,25.96,89.69,32.65,91.16,89.78,98.34,33.19,95.28,...,25.97,87.41,31.94,94.26,38.48,94.51,46.32,96.42,50.79,98.27


In [49]:
text1 = "Aarthi, as a Persistent Hiker, you bring a calm, balanced approach to your SAT preparation that values steady progress and emotional well-being. Your thoughtful and deliberate mindset helps you maintain harmony in your studies, allowing you to stay resilient even when challenges arise. You naturally prioritize mental health and sustainable growth, which is a powerful strength in managing long-term goals. At times, you might find yourself avoiding intense academic pressure or difficult topics, preferring comfort over stretch, but this awareness is the first step toward embracing growth opportunities. With gentle encouragement to take on micro-challenges, you can continue to build momentum while preserving your inner balance."
text2 = "Aarthi, as a Persistent Hiker, you embody a balanced and thoughtful approach to learning that values steady progress and emotional well-being. Your calm and deliberate mindset helps you maintain harmony in your studies, allowing you to navigate academic challenges with maturity and self-awareness. While you naturally avoid stress and prefer comfort, this same tendency can sometimes hold you back from pushing beyond your limits. Embracing small, manageable challenges will help you grow without disrupting your inner balance, turning moments of discomfort into meaningful steps forward."

In [50]:
# t1="have you been to the new restaurant in town"
# t2="have you visited the new eatery in the city"
e1 = get_embedding(text1)
e2 = get_embedding(text2)
cos2 = TfidfVectorizer().fit_transform([text1, text2])
print("\n\ncosine:", cosine_similarity(cos2[0:1], cos2[1:2])[0][0])
cos = compute_cosine_from_embeddings(e1, e2)
print("\ncosine:", cos)



cosine: 0.5585502046006118

cosine: 0.9443697862511105
